In [37]:
import os
import glob
import pandas as pd
import sktime
import numpy as np

In [2]:
sktime.__version__

'0.17.1'

In [3]:
basePath = "D:\\FinProjII\\EyeTracking\\Recording Data\\*"

In [4]:
dataList = glob.glob(basePath)
dataList = dataList[:-1]

In [5]:
identifier = [item.split("\\")[-1][-1] for item in dataList]

In [6]:
# Subjects with a code ending in 1 or 2 were reading disabled.
# Subjects with a code ending in 3 or 4 were controls. 
# Subjects with a code ending in 1 or 3 were male.
# Subjects with a code ending in 2 or 4 were female.
disabled = [int((item == '1') or (item == '2')) for item in identifier]
control = [int((item == '3') or (item == '4')) for item in identifier]
male = [int((item == '1') or (item == '3')) for item in identifier]
female = [int((item == '2') or (item == '4')) for item in identifier]

In [7]:
dataInfo = pd.DataFrame(list(zip(dataList, identifier, disabled, control, male, female)))

In [8]:
dataInfo.columns = ["file", "lastNumber", "dyslexic", "control", "male", "female"]

In [9]:
dataInfo.drop(["lastNumber", "control", "female"], axis = 1, inplace = True)

In [17]:
fileName = "A1R.txt"
data = pd.read_csv(dataInfo.file.to_list()[0] + "\\" + fileName, sep = "\t", decimal = ",")

In [11]:
dataInfo

,file,disabled,male
0,D:\FinProjII\EyeTracking\Recording Data\111GM3,0,1
1,D:\FinProjII\EyeTracking\Recording Data\111JA2,1,0
2,D:\FinProjII\EyeTracking\Recording Data\111RP1,1,1
3,D:\FinProjII\EyeTracking\Recording Data\112JU3,0,1
4,D:\FinProjII\EyeTracking\Recording Data\112KA1,1,1
...,...,...,...
180,D:\FinProjII\EyeTracking\Recording Data\826ES4,0,0
181,D:\FinProjII\EyeTracking\Recording Data\826PP1,1,1
182,D:\FinProjII\EyeTracking\Recording Data\826SJ1,1,1
183,D:\FinProjII\EyeTracking\Recording Data\831OK2,1,0


In [86]:
def makeTrainingData(df):
    fullDataX = []
    fullDataY = []
    for index, series in df.iterrows():
        file = series["file"]
        label = series["disabled"]
        fileData = pd.read_csv(file + "\\" + fileName, sep = "\t", decimal = ",")
        for i in range(len(fileData),2000):
            fileData.loc[i] = [0,0,0,0,0]
        fileData["LX"] = fileData["LX"].astype("float")
        fileData["LY"] = fileData["LY"].astype("float")
        fileData["RX"] = fileData["RX"].astype("float")
        fileData["RY"] = fileData["RY"].astype("float")
        fileData = fileData.drop(["T"], axis = 1)
        fullDataX.append(fileData)
        fullDataY.append(label)
    return fullDataX, np.array(fullDataY)

In [87]:
X, y = makeTrainingData(dataInfo)

In [88]:
from sklearn.model_selection import train_test_split
from sktime.classification.kernel_based import RocketClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rocket = RocketClassifier(num_kernels=2000)
rocket.fit(X_train, y_train)

RocketClassifier(num_kernels=2000)

In [89]:
y_pred = rocket.predict(X_test)

In [91]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_pred))

0.9459459459459459


In [92]:
y[3:5]

array([0, 1])

In [93]:
rocket.predict(X[3:5])

array([0, 1])

In [84]:
for i in range(len(data),2000):
    data.loc[i] = [0,0,0,0,0]